In [1]:
from __future__ import annotations

import os
import time
from pathlib import Path
from loguru import logger

# Import the client module
from llama_remote import client

# Helper function to create sample files
def create_sample_file(file_path, content):
    """Create a sample file with the given content."""
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    with open(file_path, 'w') as f:
        f.write(content)
    print(f"Created sample file: {file_path}")


In [2]:
print("\n--- Setting up clients ---")
# Creating Bob's client
bob_client = client("~/.syft_bob_config.json")
print(f"Connected as: {bob_client.client.email}")

# Creating Alice's client
alice_client = client("~/.syft_alice_config.json")
print(f"Connected as: {alice_client.client.email}")


--- Setting up clients ---
Connected as: bob@openmined.org
Connected as: alice@openmined.org


In [3]:
# 1. Create sample private data files
# We'll create sample files in both Alice and Bob's environments

# Alice's private data - medical research
alice_data_dir = str(alice_client.box.client.datasite_path) + "/private_data"
alice_files = {
    f"{alice_data_dir}/patient_data.csv": """
patient_id,age,gender,condition,treatment,outcome
P001,45,F,Hypertension,Medication A,Improved
P002,67,M,Diabetes,Medication B,Stable
P003,33,F,Anxiety,Therapy C,Significantly Improved
P004,52,M,Arthritis,Medication D,No Change
P005,29,F,Depression,Therapy E,Improved
""",
    f"{alice_data_dir}/research_notes.txt": """
CONFIDENTIAL RESEARCH NOTES
Date: 2025-03-01

Our preliminary findings suggest that the new treatment protocol shows a 35% improvement 
in patient outcomes compared to the standard approach. Side effects were minimal 
and reported in only 7% of participants.

Key observations:
- Treatment efficacy increases with consistent administration
- Age group 40-55 showed the most significant improvement
- No correlation found between gender and treatment response

Next steps: Expand trial to 200 additional patients and collect 6-month follow-up data.
"""
}

In [4]:

# Bob's private data - business metrics
bob_data_dir = str(bob_client.box.client.datasite_path) + "/company_data"
bob_files = {
    f"{bob_data_dir}/financial_metrics.csv": """
quarter,revenue,expenses,profit,growth_rate
Q1-2024,1250000,875000,375000,0.05
Q2-2024,1375000,950000,425000,0.08
Q3-2024,1450000,925000,525000,0.12
Q4-2024,1650000,1050000,600000,0.15
Q1-2025,1825000,1150000,675000,0.11
""",
    f"{bob_data_dir}/strategic_plan.txt": """
CONFIDENTIAL: STRATEGIC GROWTH PLAN 2025-2027

Vision: Achieve 30% market share by end of 2027
- Q2 2025: Launch new product line targeting SMB sector
- Q4 2025: Expand operations to Asian markets
- Q2 2026: Acquire competitor XYZ Technologies 
- Q4 2026: Roll out enterprise solution with advanced AI capabilities
- Q2 2027: IPO preparation and valuation assessment

Budget allocation:
- R&D: 25%
- Marketing: 30%
- Operations: 35%
- Acquisitions: 10%

Expected outcomes:
- Revenue growth: 25% YoY
- New enterprise clients: 15-20 per quarter
- Net profit margin: Increase from 18% to 24%
"""
}

In [5]:
# Create the sample files
for file_path, content in alice_files.items():
    create_sample_file(file_path, content)

for file_path, content in bob_files.items():
    create_sample_file(file_path, content)


Created sample file: /Users/atrask/Desktop/SyftBoxAlice/datasites/alice@openmined.org/private_data/patient_data.csv
Created sample file: /Users/atrask/Desktop/SyftBoxAlice/datasites/alice@openmined.org/private_data/research_notes.txt
Created sample file: /Users/atrask/Desktop/SyftBoxBob/datasites/bob@openmined.org/company_data/financial_metrics.csv
Created sample file: /Users/atrask/Desktop/SyftBoxBob/datasites/bob@openmined.org/company_data/strategic_plan.txt


In [6]:
print("\n--- Setting file permissions ---")
# Set permissions for Alice to access Bob's financial metrics
alice_permissions = bob_client.set_file_permissions(
    to_email=bob_client.client.email,  # Bob setting permissions on his own server
    user_email=alice_client.client.email,  # For Alice
    file_paths=[f"{bob_data_dir}/financial_metrics.csv"],  # Only allow access to this file
    operation="set"
)


--- Setting file permissions ---


In [7]:
print(f"Set Alice's permissions on Bob's server: {alice_permissions.success}")
print(f"Alice can access: {alice_permissions.allowed_files}")

Set Alice's permissions on Bob's server: True
Alice can access: ['/Users/atrask/Desktop/SyftBoxBob/datasites/bob@openmined.org/company_data/financial_metrics.csv']


In [8]:
# Set permissions for Bob to access Alice's patient data
bob_permissions = alice_client.set_file_permissions(
    to_email=alice_client.client.email,  # Alice setting permissions on her own server
    user_email=bob_client.client.email,  # For Bob
    file_paths=[f"{alice_data_dir}/patient_data.csv"],  # Only allow access to this file
    operation="set"
)
print(f"Set Bob's permissions on Alice's server: {bob_permissions.success}")
print(f"Bob can access: {bob_permissions.allowed_files}")

Set Bob's permissions on Alice's server: True
Bob can access: ['/Users/atrask/Desktop/SyftBoxAlice/datasites/alice@openmined.org/private_data/patient_data.csv']


In [9]:
print("\n--- Example 1: Alice queries Bob's LLM with Bob's financial data ---")
prompt1 = "Analyze the financial metrics in the file. What's the growth trend? What quarter had the highest profit?"

response1 = alice_client.generate(
    to_email=bob_client.client.email,
    model='llama3',
    prompt=prompt1,
    system="You are a financial analyst assistant. Provide concise insights based on the data.",
    temperature=0.7,
    files=[f"{bob_data_dir}/financial_metrics.csv"]  # Including Bob's file in the context
)


--- Example 1: Alice queries Bob's LLM with Bob's financial data ---


In [10]:
print(f"\nResponse from Bob's LLM:\n{'-'*40}\n{response1.response}\n{'-'*40}")



Response from Bob's LLM:
----------------------------------------
Based on the provided data, here are my concise insights:

**Growth Trend:** The company's revenue and profit have been growing steadily over the past year (2024). The growth rate has also increased over time, with a high of 0.15 in Q4-2024. This suggests a positive trend in the company's financial performance.

**Highest Profit Quarter:** Based on the data, Q4-2024 had the highest profit at $600,000, which is a 33% increase from the previous quarter ($450,000). This indicates that the company performed well in the fourth quarter of 2024.
----------------------------------------


In [11]:
print("\n--- Example 2: Bob queries Alice's LLM with Alice's patient data ---")
prompt2 = "Summarize the patient data in the file. What conditions are being treated? What's the age distribution?"

response2 = bob_client.generate(
    to_email=alice_client.client.email,
    model='llama3',
    prompt=prompt2,
    system="You are a healthcare research assistant. Provide analytical insights based on the data.",
    temperature=0.7,
    files=[f"{alice_data_dir}/patient_data.csv"]  # Including Alice's file in the context
)

print(f"\nResponse from Alice's LLM:\n{'-'*40}\n{response2.response}\n{'-'*40}")


--- Example 2: Bob queries Alice's LLM with Alice's patient data ---

Response from Alice's LLM:
----------------------------------------
Based on the provided patient data, here's a summary:

**Conditions being treated:**

1. Hypertension (P001)
2. Diabetes (P002)
3. Anxiety (P003)
4. Arthritis (P004)
5. Depression (P005)

These conditions are all chronic and require ongoing medical attention.

**Age distribution:**

The age range of the patients is 29-67 years old, with a mean age of approximately 44.5 years.

Notable observations:

* The youngest patient is 29 years old (P005), who has depression.
* The oldest patient is 67 years old (P002), who has diabetes.
* There are more female patients (3) than male patients (2).
* All patients received either medication or therapy as a treatment option, with no surgical interventions reported.

This summary provides an overview of the patient data, highlighting the conditions being treated and the age distribution. Further analysis can be pe

In [12]:
print("\n--- Example 3: Attempting to access unauthorized file ---")
# Alice tries to access Bob's strategic plan (which she doesn't have permission for)
prompt3 = "Summarize the strategic plan in the file."

response3 = alice_client.generate(
    to_email=bob_client.client.email,
    model='llama3',
    prompt=prompt3,
    system="You are a business analyst. Provide insights on the strategic plan.",
    temperature=0.7,
    files=[f"{bob_data_dir}/strategic_plan.txt"]  # Trying to access unauthorized file
)

print(f"\nResponse when trying to access unauthorized file:\n{'-'*40}\n{response3.response}\n{'-'*40}")



--- Example 3: Attempting to access unauthorized file ---

Response when trying to access unauthorized file:
----------------------------------------
I'm happy to help! However, I don't see any file provided for me to summarize. Could you please share the strategic plan document with me? Once I review it, I'll be happy to provide you with a summary of the key points and insights.
----------------------------------------


In [13]:
print("\n--- Example 4: Adding more permissions ---")
# Add permission for Alice to see the strategic plan too
alice_more_permissions = bob_client.set_file_permissions(
    to_email=bob_client.client.email,
    user_email=alice_client.client.email,
    file_paths=[f"{bob_data_dir}/strategic_plan.txt"],  
    operation="add"  # Add to existing permissions instead of replacing
)


--- Example 4: Adding more permissions ---


In [14]:

print(f"Updated Alice's permissions: {alice_more_permissions.success}")
print(f"Alice can now access: {alice_more_permissions.allowed_files}")


Updated Alice's permissions: True
Alice can now access: ['/Users/atrask/Desktop/SyftBoxBob/datasites/bob@openmined.org/company_data/financial_metrics.csv', '/Users/atrask/Desktop/SyftBoxBob/datasites/bob@openmined.org/company_data/strategic_plan.txt']


In [15]:
# Now Alice tries again with the newly granted permission
prompt4 = "Analyze the strategic plan. What are the key milestones and expected outcomes?"

response4 = alice_client.generate(
    to_email=bob_client.client.email,
    model='llama3',
    prompt=prompt4,
    system="You are a business strategy consultant. Provide insights on the plan.",
    temperature=0.7,
    files=[f"{bob_data_dir}/strategic_plan.txt"]  # Now should work
)

print(f"\nResponse after permission granted:\n{'-'*40}\n{response4.response}\n{'-'*40}")


Response after permission granted:
----------------------------------------
Based on the provided information, I will analyze the strategic plan for the company.

**Strategic Plan:**

The company aims to increase its market share by 15% within the next two years through a combination of organic growth and strategic acquisitions.

**Key Milestones:**

1. **Year 1 (2023-2024):**
	* Conduct market research and analysis to identify new business opportunities and potential acquisition targets.
	* Develop a comprehensive marketing strategy to increase brand awareness and drive leads.
	* Launch a new product or service offering to expand the company's portfolio.
2. **Year 2 (2024-2025):**
	* Acquire at least two companies that complement the current business, expanding market share and capabilities.
	* Integrate acquired businesses into the existing operations, streamlining processes and reducing costs.
	* Continue to invest in marketing efforts and expand the sales team to capitalize on new

In [16]:
# # Clean up sample files
# print("\n--- Cleaning up sample files ---")
# for file_path in alice_files:
#     if os.path.exists(file_path):
#         os.remove(file_path)
#         print(f"Removed file: {file_path}")

# for file_path in bob_files:
#     if os.path.exists(file_path):
#         os.remove(file_path)
#         print(f"Removed file: {file_path}")